In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns

train = pd.read_csv('../sale-prediction/sales_train.csv', parse_dates=['date'])
test = pd.read_csv('../sale-prediction/test.csv')
sub = pd.read_csv('../sale-prediction/sample_submission.csv')

item_categories = pd.read_csv('../sale-prediction/item_categories.csv')
items = pd.read_csv('../sale-prediction/items.csv')
shops = pd.read_csv('../sale-prediction/shops.csv')

In [3]:
shops['shop_name'].to_csv('shop_name.csv', index=False)

In [4]:
shop_name_en = pd.read_csv('../sale-prediction/shop_name_en.csv')
shops['shop_name_en'] = shop_name_en
shops['shop_name_en'][0] = shops['shop_name_en'][0].replace('! ','')
shops['shop_name_en'][1] = shops['shop_name_en'][1].replace('! ','')
def get_city(x):
    return x.split(' ')[0]

shops['city'] = shops.shop_name_en.apply(get_city)
shops['city'][[34,35]] = 'N.Novgorod'
shop_city = shops[['shop_id','city']]

C:\Users\user\Anaconda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\user\Anaconda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\user\Anaconda\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-v

In [8]:
item_category_name_en = pd.read_csv('../sale-prediction/item_category_name_en.csv')
item_categories['item_category_name_en'] = item_category_name_en
item_categories.head()

,item_category_name,item_category_id,item_category_name_en
0,PC - Гарнитуры/Наушники,0,PC - Headsets / Headphones
1,Аксессуары - PS2,1,Accessories - PS2
2,Аксессуары - PS3,2,Accessories - PS3
3,Аксессуары - PS4,3,Accessories - PS4
4,Аксессуары - PSP,4,Accessories - PSP


In [5]:
train['year'] = train.date.apply(lambda x: x.year)
train['total_price'] = train.item_price * train.item_cnt_day
train['week_day'] = train['date'].apply(lambda x: x.dayofweek)
train = train.merge(shop_city, on='shop_id')
def get_buy_or_sale(x):
    return 'sale' if x > 0 else 'buy'
    
train['sale_or_buy'] = train.item_cnt_day.apply(get_buy_or_sale)
train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,year,total_price,week_day,city,sale_or_buy
0,2013-02-01,0,59,22154,999.0,1.0,2013,999.0,4,Yaroslavl,sale
1,2013-10-01,0,59,22151,399.0,1.0,2013,399.0,1,Yaroslavl,sale
2,2013-04-01,0,59,5603,699.0,1.0,2013,699.0,0,Yaroslavl,sale
3,2013-01-19,0,59,5587,199.0,2.0,2013,398.0,5,Yaroslavl,sale
4,2013-01-31,0,59,5613,5571.0,1.0,2013,5571.0,3,Yaroslavl,sale


In [6]:
in_test_items = test.item_id.unique()
def get_in_test(x):
    if x in in_test_items:
        return 1
    else:
        return 0
    
train["in_test_item"] = train.item_id.apply(get_in_test)


in_test_shop = test.shop_id.unique()
def get_in_test(x):
    if x in in_test_shop:
        return 1
    else:
        return 0
    
train["in_test_shop"] = train.shop_id.apply(get_in_test)

In [7]:
train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,year,total_price,week_day,city,sale_or_buy,in_test_item,in_test_shop
0,2013-02-01,0,59,22154,999.0,1.0,2013,999.0,4,Yaroslavl,sale,1,1
1,2013-10-01,0,59,22151,399.0,1.0,2013,399.0,1,Yaroslavl,sale,0,1
2,2013-04-01,0,59,5603,699.0,1.0,2013,699.0,0,Yaroslavl,sale,0,1
3,2013-01-19,0,59,5587,199.0,2.0,2013,398.0,5,Yaroslavl,sale,0,1
4,2013-01-31,0,59,5613,5571.0,1.0,2013,5571.0,3,Yaroslavl,sale,0,1


In [9]:
train.loc[train.shop_id == 0, 'shop_id'] = 57
test.loc[test.shop_id == 0, 'shop_id'] = 57

train.loc[train.shop_id == 1, 'shop_id'] = 58
test.loc[test.shop_id == 1, 'shop_id'] = 58

train.loc[train.shop_id == 10, 'shop_id'] = 11
test.loc[test.shop_id == 10, 'shop_id'] = 11